In [2]:
%matplotlib qt

import time
import numpy as np
from skimage.filters import gaussian_filter

from tifffile import imread, imsave
import cPickle as pickle

from spimagine import volshow
from spimagine import EllipsoidMesh, Mesh

%reload_ext autoreload
%autoreload 2

from netsurface3d import NetSurf3d

<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


/Users/jug/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pyopencl/__init__.py:207: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


could not open /Users/jug/.spimagine
<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


# Load dataset and show in Spimagine 

In [3]:
filename = '/Users/jug/Dropbox/ProjectGoldstein/2016-09-22_fromMark/092216_PH-mCherry-02_w2SD mCherry_t1.TIF'
image = imread(filename)
print 'Image dimensions: ', image.shape

volwin = volshow(image, stackUnits = [1.,1.,5.], raise_window=False)
print 'Dataset now visible in separate Spimagine window...'

Image dimensions:  (31, 512, 512)
Dataset now visible in separate Spimagine window...


# Load vectors sampling unit sphere 

In [4]:
# load pickeled unit sphere sampling
with open('sphere_sampling.pkl','r') as f:
    dictSphereData = pickle.load(f)
    
# sampling parameters
vectors = dictSphereData['points']
neighbors = dictSphereData['neighbors']
neighbors_of = dictSphereData['neighbors_of']

# Do segmentations using NetSurf3d

In [15]:
cell1 = NetSurf3d( vectors, neighbors_of, K=30, max_delta_k=4 )
cell1_center = (110,300,15)

cell2 = NetSurf3d( vectors, neighbors_of, K=30, max_delta_k=4 )
cell2_center = (110,350,15)

In [16]:
max_rs = (image.shape[2] / 6,
          image.shape[1] / 6,
          image.shape[0] / 3)

optimum = cell1.apply_to( image, cell1_center, max_rs )
print 'Optimum energy cell 1: ', optimum
ins, outs = cell1.get_counts()
print 'Nodes in/out: ', ins, outs

optimum = cell2.apply_to( image, cell2_center, max_rs )
print 'Optimum energy cell 2: ', optimum
ins, outs = cell2.get_counts()
print 'Nodes in/out: ', ins, outs

Optimum energy cell 1:  95414.0
Nodes in/out:  8351 7009
Optimum energy cell 2:  93564.0
Nodes in/out:  7743 7617


In [17]:
green = (.7,1.,.3)
turq = (.3,1.,.7)
orange = (.5,.3,.2)
red = (.5,.2,.2)

In [34]:
#remove all meshes
while len(volwin.glWidget.meshes)>0:
    volwin.glWidget.meshes.pop(0)
volwin.glWidget.refresh()

mesh1 = cell1.create_surface_mesh( facecolor=green )
volwin.glWidget.add_mesh( mesh1 )

# mesh_center = EllipsoidMesh(rs=cell1.norm_radii((3,3,.5),image.shape), 
#                             pos=cell1.norm_coords(center,image.shape), 
#                             facecolor=green, alpha=.5)
# volwin.glWidget.add_mesh(mesh_center)

In [35]:
mesh2 = cell2.create_surface_mesh( facecolor=red )

# for i in range(5):
#     volwin.glWidget.add_mesh( mesh2 )
#     volwin.glWidget.refresh()
#     time.sleep(1)

#     volwin.glWidget.meshes.pop(-1)
#     volwin.glWidget.refresh()
#     time.sleep(1)
    
volwin.glWidget.add_mesh( mesh2 )
volwin.glWidget.refresh()

# Measure Volume

In [ ]:
#idea 1: compute volume of all pyramids defines by center point and triangle I also used for cell mesh

#idea 2: filter all points between to results that are closer then come value d. compute area of all 
#        triangles defines by those points - touching surface!